In [1]:
from PIL import Image, ImageFont, ImageDraw, ImageOps
import os
IS_GDRIVE = False

In [2]:
def get_text_dimensions(text_string, font):
    # https://stackoverflow.com/a/46220683/9263761
    ascent, descent = font.getmetrics()

    text_width = font.getmask(text_string).getbbox()[2]
    text_height = font.getmask(text_string).getbbox()[3] + descent

    return (text_width, text_height)

# You can change the join date by simply setting the parameter for it

In [3]:
if IS_GDRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

In [14]:
def decide_input_path(filename):
    return f"/content/drive/MyDrive/AutoIDCardGenerator/{filename}" if IS_GDRIVE else f"{os.getcwd()}/{filename}"

def decide_output_path(filename):
    return f"/content/{filename}" if IS_GDRIVE else f"{os.getcwd()}/{filename}"

In [5]:
"""
Most files are jpeg, png but the sheet has jpg extension. This gives an error.
"""
def search_right_img_file(filename):
    """
    params:
        filname (string): the name of the file to correct its extension
    """
    ext = ""
    exts = ["jpg", "png", "jpeg"]
    for e in exts:
        path = f"{os.getcwd()}/imgs/{filename}.{e}" if not IS_GDRIVE else f"/content/drive/MyDrive/AutoIDCardGenerator/imgs/{filename}.{e}"
        if os.path.exists(path):
            ext = f"imgs/{filename}.{e}" if not IS_GDRIVE else f"/content/drive/MyDrive/AutoIDCardGenerator/imgs/{filename}.{e}"
            break
    
    return ext

In [20]:
def setData(name,designation,id,bg,email,image_url,join_date="09/11/2022"):
    mid=350
    id_card_filepath = decide_input_path("idCard.png")
    idCard=Image.open(id_card_filepath)
    
    try:
        img=Image.open(image_url).convert("RGBA")
    except:
        print(f"\033[91m Image for ID: {id} not found. Problem with image_url: {image_url}")
        return
    
    font_bold = decide_input_path("lato/Lato-Bold.ttf")
    font_regular = decide_input_path("lato/Lato-Regular.ttf")
    titleFont=ImageFont.truetype(font_bold,37)
    nameFont=ImageFont.truetype(font_regular,30)
    idFont=ImageFont.truetype(font_regular,25)
    smallFont=ImageFont.truetype(font_regular,25)
    image_editable = ImageDraw.Draw(idCard)

    img = ImageOps.fit(img, (340, 340))
    img = ImageOps.exif_transpose(img)

    bigsize = (img.size[0] * 3, img.size[1] * 3)
    mask = Image.new('L', bigsize, 0)
    draw = ImageDraw.Draw(mask) 
    draw.ellipse((0, 0) + bigsize, fill=255)
    mask = mask.resize(img.size, Image.ANTIALIAS)
    img.putalpha(mask)

    output = ImageOps.fit(img, mask.size, centering=(0.5, 0.5))
    output.putalpha(mask)
    idCard.paste(img, (190,180), img)
    
    image_editable.text(((750-get_text_dimensions(name,titleFont)[0])/2,550), name, (0, 0, 0), font=titleFont)
    image_editable.text(((750-get_text_dimensions(designation,titleFont)[0])/2,550+50),designation,(0,0,0),font=titleFont)
    image_editable.text((260,717),id,(0,0,0),font=idFont)
    image_editable.text((259,767),email,(0,0,0),font=idFont)
    image_editable.text((550,710),bg,(0,0,0),font=nameFont)
    image_editable.text((335,838),join_date,(0,0,0),font=smallFont)
    saved_file_path = decide_output_path("ids/"+id+".png")
    idCard.save(saved_file_path) 
    print(f"\033[92m Successfully printed image of ID: {id}")

In [7]:
import pandas as pd
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\ml_projects\data_pre\env\python.exe -m pip install --upgrade pip' command.


#Run the file using generate

In [8]:
def generate(filepath, test=False, test_range=10):
    filepath = filepath if not IS_GDRIVE else f"/content/drive/MyDrive/AutoIDCardGenerator/{filepath}"
    data=pd.read_excel(filepath)
    try:
        os.mkdir("ids")
    except:
        print("Folder Already Exists")
        
    length = test_range if test else len(data)
    for i in range(length):
        name=str(data.loc[i,"Name"]).upper()
        designation=str(data.loc[i,"Member Type"])
        id=str(data.loc[i,"ID"])
        id="0"+id if len(id)<9 else id
        bg=str(data.loc[i,"Blood Group"])
        bg=" " if bg=="nan" else str(bg).upper()
        email=str(data.loc[i,"Email Address"])
        ext = str(data.loc[i, "Extension"])
        image_url = search_right_img_file(id)
        setData(name,designation,id,bg,email,image_url)
    print("\033[94m All Done!")

In [21]:
# Test on a sample of 10 data
generate("reg.xlsx", True, 10)

Folder Already Exists
 Successfully printed image of ID: 0112230867
 Successfully printed image of ID: 0112230918
 Successfully printed image of ID: 0112230122
 Successfully printed image of ID: 0112230310
 Successfully printed image of ID: 0112230150
 Successfully printed image of ID: 0112230233
 Successfully printed image of ID: 011222112
 Successfully printed image of ID: 0112230132
 Successfully printed image of ID: 0112230358
 Successfully printed image of ID: 0112230899
 All Done!


In [10]:
# Full generate
generate("reg.xlsx")